-----
# cartoGRAPHs - FEATURE MODULATION
-----

In [1]:
from cartoGRAPHs import * 

/Users/chris/Desktop/Github/cartoGRAPHs/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-20 19:21:47.874718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


DEBUG: in init: import done


_____
# DEFINE NETWORK + MATRICES
_____

In [2]:
G = nx.cycle_graph(21) #9 # 21 

l_features = ['nodeID:'+str(i) for i in list(G.nodes())]
d_features = dict(zip(G.nodes(),l_features))

### FUNCTIONAL

In [3]:
scale = 1
val = 0

rows = len(list(G.nodes()))
feat_one = [(val) if i%3 else (scale) for i in range(rows)]
feat_two = [(val) if i%2 or feat_one[i]==scale in feat_one else (scale) for i in range(rows)]
feat_three = [(scale) if feat_one[i]==val and feat_two[i]==val else val for i in range(rows)]
feat_matrix = np.vstack((feat_one,feat_two,feat_three))

DF_functional = pd.DataFrame(feat_matrix)
DF_functional.index = [100,101,102]


# ----------------------------------
# Visual settings based on features 
# ----------------------------------
col_one = 'orange' 
col_two = 'cornflowerblue'
col_three = 'skyblue'

colors_assigned_unsorted = {}
for i,feat in enumerate(feat_matrix):
    if i == 0:
        col = col_one
    elif i == 1:
        col = col_two
    elif i == 2: 
        col = col_three
    for i,v in enumerate(feat):
        if v == 1:
            colors_assigned_unsorted[i]=col
        else:
            pass
        
colors_assigned = {key:colors_assigned_unsorted[key] for key in G.nodes()}
d_colors = colors_assigned 
color_list = list(colors_assigned.values())

first_groupname = 'feature1'
second_groupname = 'feature2'
third_groupname = 'feature3'
d_legend = dict(zip(set(d_colors.values()),[first_groupname,second_groupname,third_groupname]))

# ----------------------------------
# Node sizes 
# ----------------------------------
scale_factor = 10
size = list(draw_node_degree(G, scale_factor).values())
d_size = dict(draw_node_degree(G, scale_factor))

### STRUCTURAL 

In [5]:
r=0.9
alpha=1.0

A = nx.adjacency_matrix(G)
FM_m_array = rnd_walk_matrix2(A, r, alpha, len(G.nodes()))
DF_structural = pd.DataFrame(FM_m_array).T
DF_structural.index = list(G.nodes())
DF_structural.columns = list(G.nodes()) 

_____
# FEATURE MODULATED LAYOUT
_____

In [6]:
scalar_val = 0.5
DF_merge = feature_modulation(DF_structural,DF_functional, scalar_val)

layout_method = 'functional'
posG2D = generate_layout(G, 
                        dim = 2, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        Matrix = DF_merge
                        )

posG3D = generate_layout(G, 
                        dim = 3, 
                        layoutmethod = layout_method,
                        dimred_method='umap',
                        Matrix = DF_merge
                        )

/Users/chris/Desktop/Github/cartoGRAPHs/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/chris/Desktop/Github/cartoGRAPHs/.venv/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/chris/Desktop/Github/cartoGRAPHs/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no 

_____
# VISUALIZATION
_____

In [7]:
plot_2Dfigure(G, posG2D, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = '', 
              fname = '2Dportrait'+'_'+layout_method, 
              scheme = 'light')

plot_3Dfigure(G, posG3D, 
              d_features, 
              d_colors, 
              d_size, 
              d_legend, 
              path = '', 
              fname = '3Dportrait'+'_'+layout_method, 
              scheme = 'light')

'3Dportrait_functional.html'